In [1]:
import math as mt

In [2]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import re
from collections import namedtuple
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [3]:
#Settings
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [4]:
#Read the preprocessed Data
filepath = './CICIDS2017_preprocessed'
data = pd.read_csv(filepath)

In [5]:

data.shape

(2830743, 21)

In [6]:
data.head()

,Destination Port,Flow Duration,Fwd Packet Length Max,Bwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Mean,Fwd IAT Min,Bwd IAT Mean,Bwd IAT Std,Fwd Packets/s,Packet Length Mean,Packet Length Std,PSH Flag Count,Average Packet Size,Avg Bwd Segment Size,Subflow Fwd Packets,Init_Win_bytes_forward,Init_Win_bytes_backward,Active Std,Label_binary
0,0.000336,1.491667e-06,0.000000,0.0,0.000000e+00,1.500000e-06,0.000000,9.999999e-08,0.000000e+00,0.0,2.008032e-03,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.004440,0.003723,0.000000,0
1,0.917800,7.999999e-07,0.000000,0.0,1.250775e-07,4.000000e-07,0.000000,9.999999e-08,4.083333e-07,0.0,4.016064e-03,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.003723,0.004440,0.000000,0
2,0.001877,8.329999e-04,0.001934,0.0,0.000000e+00,8.330082e-04,0.000000,9.999999e-08,0.000000e+00,0.0,3.335101e-06,0.014384,0.0,0.0,0.018493,0.008275,0.000000,0.000000,0.000000,0.000000,0
3,0.001877,3.085833e-04,0.001934,0.0,0.000000e+00,3.085916e-04,0.000000,9.999999e-08,0.000000e+00,0.0,9.004872e-06,0.014384,0.0,0.0,0.018493,0.008275,0.000000,0.000000,0.000000,0.000000,0
4,0.000000,9.263445e-01,0.000000,0.0,2.995055e-02,1.166667e-07,0.006345,9.999999e-08,0.000000e+00,0.0,4.408007e-07,0.000000,0.0,0.0,0.000000,0.000000,0.000664,0.000000,0.000000,0.028615,0


In [5]:
#function that takes the columns of dataframe and turns them into arrays stored in a single array
def dataframe_to_arr(data): 
    data_arr = []
    row = 0
    for col in data.columns:
        row += 1
        col_arr = data[col].to_numpy()
        data_arr.append(col_arr)
    return data_arr

In [6]:
#function that gets the most redundant value in an array
def maxFreq(arr, n):
    
    res = 0
    count = 1
    
    for i in range(1,n):
        if (arr[i] == arr[res]):
            count +=1
        else:
            count -= 1
        
        if (count == 0):
            res = i
            count = 1
            
    return arr[res]

In [7]:
#function that gets the most frequent values for each of the array in a dataframe
#It returns an array of most redundant value in each of the df columns
def most_freq_nbr(data_arr):
    size = len(data_arr)
    freq_arr = []
    for i in range(0, size):
        v = data_arr[i]
        n = len(v)
        max_nbr =  maxFreq(v, n)
        freq_arr.append(max_nbr)
    return freq_arr

In [8]:
#function that gets the indices of the most redundant value from the array
from itertools import count # izip for maximum efficiency

def get_arr_index(data_arr, max_nbr):
    indices = [i for i, j in zip(count(), data_arr) if j == max_nbr] 
    return indices

In [9]:
def get_indices(data):    
    data_arr = dataframe_to_arr(data)
    max_freq_arr = most_freq_nbr(data_arr)
    indices = []
    for i in range(0,len(max_freq_arr)):
        ind = get_arr_index(data_arr[i], max_freq_arr[i])
        indices.append(ind)
    
    return indices, max_freq_arr

In [10]:
def common_values(a,b):
    a_set = set(a)
    b_set = set(b)
    
    v = list(a_set & b_set)
    if v:
        print(v)
    else:
        print("No common elements")
        
    return v

In [67]:
f_set = set([1,3,5,6])
v_set = set([1,6,9])

print(f_set, v_set)
cmon_list = list(f_set & v_set)

if cmon_list:
    print(cmon_list)

{1, 3, 5, 6} {1, 6, 9}
[1, 6]


In [13]:
indices = get_indices(data)

In [28]:
indices[0][0]

[0,
 320,
 447,
 448,
 465,
 1131,
 1132,
 1133,
 1135,
 1465,
 1466,
 1467,
 1470,
 1478,
 1479,
 1480,
 1482,
 7312,
 7321,
 7322,
 7327,
 7328,
 8647,
 8650,
 8651,
 8653,
 9949,
 9951,
 9953,
 9955,
 14077,
 14095,
 14096,
 14121,
 17989,
 18027,
 18030,
 18034,
 21535,
 21549,
 21550,
 21551,
 21707,
 21708,
 23111,
 23112,
 23113,
 23115,
 23133,
 23191,
 23200,
 23246,
 24057,
 24059,
 24060,
 24063,
 25325,
 25329,
 25330,
 25332,
 25380,
 25395,
 25396,
 25398,
 25427,
 25456,
 25457,
 25478,
 26660,
 26749,
 26750,
 26751,
 26760,
 26785,
 26788,
 26789,
 26793,
 26794,
 27071,
 27081,
 27082,
 27084,
 27724,
 27788,
 27794,
 27894,
 29976,
 29981,
 29982,
 29994,
 30038,
 30043,
 30044,
 30047,
 30048,
 30099,
 30109,
 30110,
 30123,
 30124,
 31143,
 31148,
 31154,
 31159,
 31160,
 31163,
 31164,
 31173,
 31513,
 31658,
 31660,
 31836,
 33776,
 33831,
 33835,
 33869,
 34119,
 34123,
 34124,
 34131,
 34147,
 34304,
 34307,
 34365,
 35513,
 35516,
 35517,
 35519,
 35526,
 3553

In [36]:
indices[1]

[0.0003356984817273,
 5.249999518750045e-07,
 0.0,
 0.0,
 4.002478868938438e-07,
 1.2499998541666836e-07,
 0.0,
 9.9999990000001e-08,
 0.0,
 0.0,
 0.0066666666666666,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.550460051511208e-06,
 0.0,
 0.0,
 0.0,
 0]

In [30]:
common_values(indices[0][0], indices[0][1]) 

{1724043, 1685910, 1698839, 1719712, 1726755, 1843624, 1695400, 1714346, 1723434, 1958696, 1695153, 1866421, 1685304, 174776, 1966650, 1819454, 1708356, 1713992, 1717070, 1925583, 1696341, 1697109, 1957335, 1693656, 1707483, 1720283, 1698651, 1942495, 1725793, 1726305, 1920611, 1967722, 1948522, 1709548, 1708397, 1684717, 1828588, 1947634, 1913588, 1722490, 1920894, 1694207}


In [43]:
common_values(indices[0][0], indices[0][1])

{1724043, 1685910, 1698839, 1719712, 1726755, 1843624, 1695400, 1714346, 1723434, 1958696, 1695153, 1866421, 1685304, 174776, 1966650, 1819454, 1708356, 1713992, 1717070, 1925583, 1696341, 1697109, 1957335, 1693656, 1707483, 1720283, 1698651, 1942495, 1725793, 1726305, 1920611, 1967722, 1948522, 1709548, 1708397, 1684717, 1828588, 1947634, 1913588, 1722490, 1920894, 1694207}


{174776,
 1684717,
 1685304,
 1685910,
 1693656,
 1694207,
 1695153,
 1695400,
 1696341,
 1697109,
 1698651,
 1698839,
 1707483,
 1708356,
 1708397,
 1709548,
 1713992,
 1714346,
 1717070,
 1719712,
 1720283,
 1722490,
 1723434,
 1724043,
 1725793,
 1726305,
 1726755,
 1819454,
 1828588,
 1843624,
 1866421,
 1913588,
 1920611,
 1920894,
 1925583,
 1942495,
 1947634,
 1948522,
 1957335,
 1958696,
 1966650,
 1967722}

In [31]:
data.loc[1724043].at["Destination Port"]

0.0003356984817273

In [32]:
data.loc[1724043].at["Flow Duration"]

5.249999518750045e-07

In [35]:
indices[0]

[[0,
  320,
  447,
  448,
  465,
  1131,
  1132,
  1133,
  1135,
  1465,
  1466,
  1467,
  1470,
  1478,
  1479,
  1480,
  1482,
  7312,
  7321,
  7322,
  7327,
  7328,
  8647,
  8650,
  8651,
  8653,
  9949,
  9951,
  9953,
  9955,
  14077,
  14095,
  14096,
  14121,
  17989,
  18027,
  18030,
  18034,
  21535,
  21549,
  21550,
  21551,
  21707,
  21708,
  23111,
  23112,
  23113,
  23115,
  23133,
  23191,
  23200,
  23246,
  24057,
  24059,
  24060,
  24063,
  25325,
  25329,
  25330,
  25332,
  25380,
  25395,
  25396,
  25398,
  25427,
  25456,
  25457,
  25478,
  26660,
  26749,
  26750,
  26751,
  26760,
  26785,
  26788,
  26789,
  26793,
  26794,
  27071,
  27081,
  27082,
  27084,
  27724,
  27788,
  27794,
  27894,
  29976,
  29981,
  29982,
  29994,
  30038,
  30043,
  30044,
  30047,
  30048,
  30099,
  30109,
  30110,
  30123,
  30124,
  31143,
  31148,
  31154,
  31159,
  31160,
  31163,
  31164,
  31173,
  31513,
  31658,
  31660,
  31836,
  33776,
  33831,
  33835,
  

In [43]:
freq = maxFreq(arr, n)
count = 0
for i in range(n):
    if(arr[i] == freq):
        count += 1

print("Element ", maxFreq(arr, n), " occurs ", count, " times")

Element  0.0003356984817273  occurs  16941  times


In [13]:
v = np.array([[4,6,2,6,6,6,6,6],[2,2,6,2,5,2,2,6],[2,3,3,6,3,5,3,7]])

In [14]:
v.T

array([[4, 2, 2],
       [6, 2, 3],
       [2, 6, 3],
       [6, 2, 6],
       [6, 5, 3],
       [6, 2, 5],
       [6, 2, 3],
       [6, 6, 7]])

In [11]:
df = pd.DataFrame(data = np.array([[4,6,2,6,6,6,6,6,5,4],[2,2,6,2,5,2,2,6,8,2],[2,3,3,6,3,5,3,7,1,3]]).T, 
                  columns = ['A', 'B', 'C'])

In [12]:
df

,A,B,C
0,4,2,2
1,6,2,3
2,2,6,3
3,6,2,6
4,6,5,3
5,6,2,5
6,6,2,3
7,6,6,7
8,5,8,1
9,4,2,3


In [13]:
arr_ind, max_vals = get_indices(df)

In [15]:
arr_ind

[[1, 3, 4, 5, 6, 7], [0, 1, 3, 5, 6, 9], [1, 2, 4, 6, 9]]

In [16]:
arr_ind[1]

[0, 1, 3, 5, 6, 9]

In [17]:
max_vals

[6, 2, 3]

In [19]:
cmon  = common_values(indices[0], indices[2])

[1, 4, 6]


In [20]:
common_values(df['A'], df['C'])

[2, 5, 6]


[2, 5, 6]

In [18]:
def cmon_row(indices):
    common_rows = []
    for v in range(len(indices)):

        if v == len(indices)-1:
            cmon_val = common_values(indices[v], indices[-len(indices)])
            common_rows.append(cmon_val)
        else:
            cmon_val = common_values(indices[v], indices[v+1])
            common_rows.append(cmon_val)
    
    return common_rows

In [21]:
cmn_ind = cmon_row(arr_ind)

[1, 3, 5, 6]
[1, 6, 9]
[1, 4, 6]


In [76]:
ccm = common_values(indices[0], indices[1])

[1, 3, 5, 6]


In [66]:
len(indices)

3

In [34]:
indices[-2]

[0, 1, 3, 5, 6, 9]

In [29]:
t = np.array(common_rows[0])
y = np.array(common_rows[1])
u = np.array(common_rows[2])

print('\n', t,'\n', y, '\n', u, '\n')

test = np.concatenate(t,y)


 {1, 3, 5, 6} 
 {1, 6, 9} 
 {1, 4, 6} 



TypeError: dispatcher for __array_function__ did not return an iterable

In [55]:
s = [2,3,3,6,3,5,3,7,1,3]
d = [4,6,2,6,6,6,6,6,5,4]
f = [2,2,6,2,5,2,2,6,8,2]
n = len(d)


In [57]:
maxFreq(s, n)

3